In [76]:
import numpy as np                
from numpy import linalg as LA    #For the varinici algorithm
import matplotlib.pyplot as plt   #For the gifs
import imageio                    #For the gifs
import os
from scipy.spatial import ConvexHull, convex_hull_plot_2d #To plot the boundary of each cell


In [382]:
def varinoci(dots,size_domine,partition):
    ############
    # We divide the domine following the parameters, then travel for all the mesh classifying the dots. As you can see this is 
    # a expensive way to compute the voronoid diagram. There exist a more cleaver way to do this. This is using Fortune's 
    # algorithm https://en.wikipedia.org/wiki/Fortune%27s_algorithm 
    ############
    tessellation = {}
    X = np.linspace(0, size_domine[0], partition)
    Y = np.linspace(0, size_domine[1], partition)  
    for i in range(partition):
        for j in range(partition):          
            minimo = 500  
            actual_dot = [X[i],Y[j]]
            for k in range(len(dots)):      
                value =  LA.norm(actual_dot - dots[k])
                if value < minimo:
                    minimo = value
                    k_star = k
            if k_star in tessellation.keys():
                tessellation[k_star].append(actual_dot)
            else:
                tessellation[k_star] = [actual_dot]
     
    return tessellation

In [383]:
def new_centroids(tessellation,dots,partition,size_domine):
    ############
    # We estimate the centroids of each section.
    ###########
    new_dots=[]
    h=size_domine[0]/(partition-1)
    for i in range(len(tessellation.keys())):
        xi=np.array(tessellation[i])
        #print(xi.shape)
        #hull = ConvexHull(xi)
        #print(hull.volume)
        #CI = hull.volume
        CI = xi.shape[0] 
        #print(CI)
        new_dots.append((1/CI)*np.sum(xi,axis=0))
    return np.array(new_dots)

In [516]:
def plot_tessell(N,tessellation,new_dots,itera,imagen_names):
    ################
    # To plot the tessellation with differents colors.
    ################
    cwd = os.getcwd()

    newpath = os.path.join(cwd,"imagen") #We create a new path to save the images
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    
    plt.figure(itera)
    for k in range(N):
        points=np.array(tessellation[k])
        hull = ConvexHull(points)
        a=0.5 
        x=points[hull.vertices,0]
        y=points[hull.vertices,1]
        plt.fill(x,y,c = 'C'+str(k), alpha=a)
        
        for simplex in hull.simplices:
            plt.plot(points[simplex, 0], points[simplex, 1], 'k-',linewidth = 3.1)
        
    plt.scatter(new_dots[:,0], new_dots[:,1], c = 'k' , s = 20, linewidth = 2 )   
    plt.axis('off')
    
            
    #plt.savefig(newpath+'\\Floyd_algorithm_'+str(itera)+'.png')
    save_path=os.path.join(newpath,'Floyd_algorithm_'+str(itera)+'.png') #We save the images
                           
    plt.savefig(save_path)
    plt.close()
    imagen_names.append(save_path)
    return imagen_names

In [517]:
def floyd_algoritm(Iterations,N,size_domine,partition,seed=None,plot=False,history=False):
    ###############
    # Main funtion who assambles the algorithm
    ###############
    if seed is not None:
        np.random.seed(seed)
        
    new_dots = np.random.random([N,2])*size_domine
    
    history_tessell = []
    history_dots    = [new_dots]
    imagen_names=[]

    for itera in range(Iterations):
        tessellation  =  varinoci(new_dots,size_domine,partition)
        new_dots      =  new_centroids(tessellation,new_dots,partition,size_domine)
        if plot == True:
            imagen_names = plot_tessell(N,tessellation,new_dots,itera,imagen_names)
            #print('finish plot Number:' +str(itera))
        if history==True:    
            history_tessell.append(tessellation)
            history_dots.append(new_dots)
        if itera%10==0:
            print('iteration number '+str(itera)+' executed.')
    return imagen_names, history_tessell, history_dots

In [519]:
############### SETINGS #################

size_domine=[2,2]  #Define the size of the domine (a square with a vertix in (0,0))
N_dots = 8        #How much dots in your example you need
iterations = 200    #How much iterations
plot = True       #If you need to plot or not
history = False    #If you need the history of each iteration
seed = 6           #If you coment this line, each example will to have differents dots
partition=350       #The resolution of the mesh

########### MAIN ALGORITHM ##################


imagen_names, history_tessell, history_dots=floyd_algoritm(iterations,N_dots,size_domine,partition,seed,plot,history)




iteration number 0 executed.


In [520]:
############## GIF #################
if plot is True:
    new_imagen_names=[]
    Frames=1                       #If you want an animation more slow, you can increce this parameter
    for itera in range(iterations):
        for frame in range(Frames):
            new_imagen_names.append(imagen_names[itera])
            
    gif_name='Lloyd_algorithm_P'+str(partition)+'_S'+str(seed)+'_Dim'+str(size_domine[0])+'x'+str(size_domine[1])+'_I'+str(iterations)+'_N'+str(N_dots)
    
    cwd = os.getcwd()
    
    newpath_gif = cwd + '\\gifs' 
    if not os.path.exists(newpath_gif):
        os.makedirs(newpath_gif)
    
    with imageio.get_writer(newpath_gif+'\\'+gif_name+'.gif', mode='I') as writer:
        for filename in new_imagen_names:
            image = imageio.imread(filename)
            writer.append_data(image)
